In [1]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow import keras 
from tensorflow.keras import layers
import os
from PIL import Image
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from keras.layers import Conv2D, MaxPooling2D, Dropout, GlobalAveragePooling2D, Dense, BatchNormalization, AveragePooling2D, Flatten
from sklearn import svm 
from sklearn.model_selection import GridSearchCV 
from sklearn.metrics import accuracy_score 
from sklearn.metrics import classification_report
import aspose.words as aw

## First (large) dataset

In [ ]:
'''Convert all images to PNG -- run only one time'''

def save_images_as_png(folder_path, output_folder, image_size=(150, 150)):
    for root, dirs, files in os.walk(folder_path):
        for filename in files:
            file_path = os.path.join(root, filename)
            if any(filename.endswith(extension) for extension in ['.jpg', '.jpeg', '.png', '.gif']):
                try:
                    with Image.open(file_path) as img:
                        # Create the output folder structure if it doesn't exist
                        relative_path = os.path.relpath(root, folder_path)
                        output_subfolder = os.path.join(output_folder, relative_path)
                        os.makedirs(output_subfolder, exist_ok=True)
                        # Convert and save the image as PNG
                        if img.mode == 'CMYK':
                            img = img.convert('RGB')
                        output_file_path = os.path.join(output_subfolder, filename.split('.')[0] + '.png')
                        img = img.resize(image_size)  # resizing image
                        #img = img.convert('L')  # Converting to grayscale
                        img.save(output_file_path, format='PNG')
                except Exception as e:
                    print(f"Error processing image '{file_path}': {e}")

folder_path = r"C:\Users\emili\OneDrive\Dokumente\03_Master_CBS\05_Summerterm24\01_ML and DL\Project\logo findder"
output_folder=r"C:\Users\emili\OneDrive\Dokumente\03_Master_CBS\05_Summerterm24\01_ML and DL\Project\png logo findder"
save_images_as_png(folder_path, output_folder)

In [2]:
"""Getting df with all filenames and images (missing labels at this stage) - shape = (166866,2)"""
# only take first ten photos in each brand folder
# (150,150) grayscale --> 2m 28.9s
# (150,150) rgb, with pixels as cols --> 

def load_images_from_folder(folder_path, image_size=(150, 150)):
    images = []
    
    # Recursively traverse through the folder structure
    for root, dirs, files in os.walk(folder_path):
        for filename in files:
            # Get the full path of the file
            file_path = os.path.join(root, filename)
            
            # Check if the file is an image
            if any(filename.endswith(extension) for extension in ['.jpg', '.jpeg', '.png', '.gif']):
                # Load the image using PIL (Python Imaging Library)
                try:
                    with Image.open(file_path) as img:
                        img = img.resize(image_size) #resizing image
                        img = img.convert('RGB') #Converting to gray scale
                        #img_array = np.array(img) / 255.0 # Convert the image to numpy array and normalize pixel values
                         # Extract folder name from the root path
                        label = os.path.basename(root)
                        
                        images.append({'Label': label, 'Filename': filename, 'Image': img})
                        #images.append({'Filename': filename, 'Image': img}) # Append the image array to the list
                except Exception as e:
                    print(f"Error loading image '{file_path}': {e}")
    
    # Convert the list of dictionaries to a DataFrame
    images_df = pd.DataFrame(images)
    return images_df

# Loading
png_folder_path=r"C:\Users\emili\OneDrive\Dokumente\03_Master_CBS\05_Summerterm24\01_ML and DL\Project\png logo findder"
image_df = load_images_from_folder(png_folder_path)
print('Image Dataframe: \n \n', image_df.head())

C:\Users\emili\anaconda3\Lib\site-packages\PIL\Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Error loading image 'C:\Users\emili\OneDrive\Dokumente\03_Master_CBS\05_Summerterm24\01_ML and DL\Project\png logo findder\Original Aston Martin logo\Image_30.png': Decompressed Data Too Large
Error loading image 'C:\Users\emili\OneDrive\Dokumente\03_Master_CBS\05_Summerterm24\01_ML and DL\Project\png logo findder\Original Maserati logo\Image_13.png': Decompressed Data Too Large
Error loading image 'C:\Users\emili\OneDrive\Dokumente\03_Master_CBS\05_Summerterm24\01_ML and DL\Project\png logo findder\Original Porsche logo\Image_18.png': Decompressed Data Too Large
Image Dataframe: 
 
                         Label      Filename  \
0  Original Aston Martin logo  Image_10.png   
1  Original Aston Martin logo  Image_11.png   
2  Original Aston Martin logo  Image_12.png   
3  Original Aston Martin logo  Image_13.png   
4  Original Aston Martin logo  Image_15.png   

                                               Image  
0  <PIL.Image.Image image mode=RGB size=150x150 a...  
1  <PIL.Image.Im

In [ ]:
"""Run to get an overview of the data"""

#Get an overview of the number of images per label
labelinfo = image_df['Label'].value_counts() 
print('About the Distribtion of Labels:')
print('The number of different brands is', len(labelinfo))
print('The number of images per brand ranges from',labelinfo.min(),'to', labelinfo.max())
print('The average is',labelinfo.mean().round(),'and the median is', labelinfo.median())

#Get an overivew of image characteristics (size, channels) in the dataset
#function to extract image characteristics
def get_image_characteristics(image):
    size = image.size  # Get image size (width, height)
    mode = image.mode  # Get image mode (color mode)
    return size, mode

image_sizes = []
image_modes = []
for index, row in image_df.iterrows():
    size, mode = get_image_characteristics(row['Image'])  # Assuming 'Image' is the column containing PIL images
    image_sizes.append(size)
    image_modes.append(mode)

image_characteristics_df = pd.DataFrame({'Size': image_sizes, 'Mode': image_modes})

#Print an overview of the image characteristics
print('\nAbout the Images in the Dataset:')
print('The number of different sizes is', len(image_characteristics_df['Size'].unique()))
print('Image size(s) in the dataset:', image_characteristics_df['Size'].unique())
print('The number of different color modes is', len(image_characteristics_df['Mode'].unique()))
print('Color mode(s) in the dataset:', image_characteristics_df['Mode'].unique())

#Get an overivew of the first rows of df
print('\n')
print('Quick view of dataframe:')
image_df.head()

In [ ]:
# Selecting one picture from the DataFrame
some_pic = image_df['Image'].iloc[7]

# Display the image using matplotlib
plt.imshow(some_pic, interpolation="lanczos")
plt.axis("off")
plt.show()

In [3]:
#rename the car brands
image_df['Label'] = image_df['Label'].str.replace('Original', '').str.replace('logo', '').str.replace(' ', '')
image_df

,Label,Filename,Image
0,AstonMartin,Image_10.png,<PIL.Image.Image image mode=RGB size=150x150 a...
1,AstonMartin,Image_11.png,<PIL.Image.Image image mode=RGB size=150x150 a...
2,AstonMartin,Image_12.png,<PIL.Image.Image image mode=RGB size=150x150 a...
3,AstonMartin,Image_13.png,<PIL.Image.Image image mode=RGB size=150x150 a...
4,AstonMartin,Image_15.png,<PIL.Image.Image image mode=RGB size=150x150 a...
...,...,...,...
364,Porsche,Image_5.png,<PIL.Image.Image image mode=RGB size=150x150 a...
365,Porsche,Image_6.png,<PIL.Image.Image image mode=RGB size=150x150 a...
366,Porsche,Image_7.png,<PIL.Image.Image image mode=RGB size=150x150 a...
367,Porsche,Image_8.png,<PIL.Image.Image image mode=RGB size=150x150 a...


## Second (small) dataset

In [14]:
'''Read in the second data set'''

image_directory = r"C:\Users\emili\OneDrive\Dokumente\03_Master_CBS\05_Summerterm24\01_ML and DL\Project\USA Car Logos"

image_filenames = [f for f in os.listdir(image_directory) if os.path.isfile(os.path.join(image_directory, f))]

# Function to process images and store them in a DataFrame
def process_images(image_directory, image_filenames):
    image_data = []  # List to store image data

    for filename in image_filenames:
        try:
            image_path = os.path.join(image_directory, filename)  # Construct the full path to the image file
            img = Image.open(image_path)  # Open the image using Pillow (PIL)
            img = img.convert('RGB') #Converting to gray scale
            img_resized = img.resize((150, 150), Image.LANCZOS)  # Resize the image to 100x100 pixels
            # img_array = np.array(img_resized) # Convert image to numpy array if needed
            image_data.append({'Filename': filename, 'Image': img_resized})

        except Exception as e:
            # Handle errors (e.g., if the image cannot be processed)
            print(f"Error processing image {filename}: {e}")

    # Convert the list of dictionaries to a DataFrame
    image_df = pd.DataFrame(image_data)
    return image_df

image2_df=process_images(image_directory, image_filenames)
image2_df

,Filename,Image
0,AstonMartin.png,<PIL.Image.Image image mode=RGB size=150x150 a...
1,Audi.png,<PIL.Image.Image image mode=RGB size=150x150 a...
2,Bentley.png,<PIL.Image.Image image mode=RGB size=150x150 a...
3,Ferrari.png,<PIL.Image.Image image mode=RGB size=150x150 a...
4,FiskerAutomotive.png,<PIL.Image.Image image mode=RGB size=150x150 a...
5,Ford.png,<PIL.Image.Image image mode=RGB size=150x150 a...
6,Lamborghini.png,<PIL.Image.Image image mode=RGB size=150x150 a...
7,LotusCars.png,<PIL.Image.Image image mode=RGB size=150x150 a...
8,Maserati.png,<PIL.Image.Image image mode=RGB size=150x150 a...
9,Mercedes-AMG.png,<PIL.Image.Image image mode=RGB size=150x150 a...


In [ ]:
"""Run to get an overview of this dataset as well"""

#Get an overview of the number of images per label
labelinfo2 = image2_df['Filename'].value_counts() 
print('About the Distribtion of Labels:')
print('The number of different brands is', len(labelinfo2))
print('The number of images per brand ranges from',labelinfo2.min(),'to', labelinfo2.max())
print('The average is',labelinfo2.mean().round(),'and the median is', labelinfo2.median())

#Get an overivew of image characteristics (size, channels) in the dataset
#function to extract image characteristics
def get_image_characteristics(image):
    size = image.size  # Get image size (width, height)
    mode = image.mode  # Get image mode (color mode)
    return size, mode

image_sizes = []
image_modes = []
for index, row in image2_df.iterrows():
    size, mode = get_image_characteristics(row['Image'])  # Assuming 'Image' is the column containing PIL images
    image_sizes.append(size)
    image_modes.append(mode)

image_characteristics_df2 = pd.DataFrame({'Size': image_sizes, 'Mode': image_modes})

#Print an overview of the image characteristics
print('\nAbout the Images in the Dataset:')
print('The number of different sizes is', len(image_characteristics_df2['Size'].unique()))
print('Image size(s) in the dataset:', image_characteristics_df2['Size'].unique())
print('The number of different color modes is', len(image_characteristics_df2['Mode'].unique()))
print('Color mode(s) in the dataset:', image_characteristics_df2['Mode'].unique())

#Get an overivew of the first rows of df
print('\n')
print('Quick view of dataframe:')
image2_df.head()

In [15]:
#rename files
image2_df['Label'] = image2_df['Filename'].apply(lambda x: x.split('.')[0])
image2_df

,Filename,Image,Label
0,AstonMartin.png,<PIL.Image.Image image mode=RGB size=150x150 a...,AstonMartin
1,Audi.png,<PIL.Image.Image image mode=RGB size=150x150 a...,Audi
2,Bentley.png,<PIL.Image.Image image mode=RGB size=150x150 a...,Bentley
3,Ferrari.png,<PIL.Image.Image image mode=RGB size=150x150 a...,Ferrari
4,FiskerAutomotive.png,<PIL.Image.Image image mode=RGB size=150x150 a...,FiskerAutomotive
5,Ford.png,<PIL.Image.Image image mode=RGB size=150x150 a...,Ford
6,Lamborghini.png,<PIL.Image.Image image mode=RGB size=150x150 a...,Lamborghini
7,LotusCars.png,<PIL.Image.Image image mode=RGB size=150x150 a...,LotusCars
8,Maserati.png,<PIL.Image.Image image mode=RGB size=150x150 a...,Maserati
9,Mercedes-AMG.png,<PIL.Image.Image image mode=RGB size=150x150 a...,Mercedes-AMG


## Data Augmentation

In [6]:
#augment the data with tf (more common is to add these layers into the model, but what about SVM then?)
img_size=150

augment = tf.keras.Sequential([
    layers.Resizing(img_size, img_size), #to make size every pic is 150x150 (necessary?)
    layers.RandomFlip(mode='horizontal_and_vertical'), #flipping it
    layers.RandomZoom(0.2),
    layers.RandomBrightness([-0.3,0.3]),
    layers.RandomRotation(0.1), #randomly rotate it
    layers.Rescaling(1./255) #we aslo rescale it later so maybe not?
])

In [ ]:
#plot one image
some_pic_tf = tf.cast(tf.expand_dims(some_pic, 0), tf.float32)

plt.figure(figsize=(15, 15))

for i in range(9):
    augmented_image = augment(some_pic_tf)
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(augmented_image[0])
    plt.axis("off")

In [7]:
'''This function will apply the augmentation mechanisms'''

def apply_aug(img, aug, filename, Label, num_aug_images=7): #maybe even more than seven
    img = tf.convert_to_tensor(img)
    img = tf.expand_dims(img, axis=0)
    augmented_img = aug(img)[0] 
    augmented_images = [augmented_img for _ in range(num_aug_images)]
    
    # Convert the augmented images to PIL Image objects if needed
    #augmented_images = [Image.fromarray((augmented_img.numpy() * 255).astype('uint8')) for augmented_img in augmented_images]
    
    # Create a DataFrame to store the augmented images
    data = {'Filename': [filename] * len(augmented_images), 'Label': [Label]* len(augmented_images),'Augmented_Image': augmented_images}
    df = pd.DataFrame(data)
    
    return df

def augment_images(df, label, name, image):
    '''This function actually applies the augmentation to images in the DataFrame'''
    dataframes = []  #Creating a list to store the augmented images

    for index, row in df.iterrows(): #Loop through each row in the data frame
        Label = row[label]
        filename = row[name]
        
        #Find the original image in the input data frame 
        original_image_row = df[df[name] == filename].iloc[0]
        original_image = original_image_row[image]
        
        #Apply augmentation of the previous function to the original image
        augmented_images_df = apply_aug(original_image, augment, filename, Label)
        
        #Append the augmented images to the initilaized list
        dataframes.append(augmented_images_df)

    #transform the list to a new data frame
    augmented_df = pd.concat(dataframes, ignore_index=True)
    
    return augmented_df

In [8]:
#apply augmentation to our data
augmented_df=augment_images(image_df, 'Label', 'Filename', 'Image' )
augmented_df

,Filename,Label,Augmented_Image
0,Image_10.png,AstonMartin,"(((tf.Tensor(0.16604683, shape=(), dtype=float..."
1,Image_10.png,AstonMartin,"(((tf.Tensor(0.16604683, shape=(), dtype=float..."
2,Image_10.png,AstonMartin,"(((tf.Tensor(0.16604683, shape=(), dtype=float..."
3,Image_10.png,AstonMartin,"(((tf.Tensor(0.16604683, shape=(), dtype=float..."
4,Image_10.png,AstonMartin,"(((tf.Tensor(0.16604683, shape=(), dtype=float..."
...,...,...,...
2578,Image_9.png,Porsche,"(((tf.Tensor(0.44357595, shape=(), dtype=float..."
2579,Image_9.png,Porsche,"(((tf.Tensor(0.44357595, shape=(), dtype=float..."
2580,Image_9.png,Porsche,"(((tf.Tensor(0.44357595, shape=(), dtype=float..."
2581,Image_9.png,Porsche,"(((tf.Tensor(0.44357595, shape=(), dtype=float..."


In [16]:
#apply augmentation to our data
augmented2_df=augment_images(image2_df, 'Label', 'Filename', 'Image' )
augmented2_df

,Filename,Label,Augmented_Image
0,AstonMartin.png,AstonMartin,"(((tf.Tensor(0.20168391, shape=(), dtype=float..."
1,AstonMartin.png,AstonMartin,"(((tf.Tensor(0.20168391, shape=(), dtype=float..."
2,AstonMartin.png,AstonMartin,"(((tf.Tensor(0.20168391, shape=(), dtype=float..."
3,AstonMartin.png,AstonMartin,"(((tf.Tensor(0.20168391, shape=(), dtype=float..."
4,AstonMartin.png,AstonMartin,"(((tf.Tensor(0.20168391, shape=(), dtype=float..."
...,...,...,...
72,Porsche.png,Porsche,"(((tf.Tensor(1.0, shape=(), dtype=float32), tf..."
73,Porsche.png,Porsche,"(((tf.Tensor(1.0, shape=(), dtype=float32), tf..."
74,Porsche.png,Porsche,"(((tf.Tensor(1.0, shape=(), dtype=float32), tf..."
75,Porsche.png,Porsche,"(((tf.Tensor(1.0, shape=(), dtype=float32), tf..."


## Merge and Encode the two dataframes

In [17]:
augmented_df_full = pd.concat([augmented_df, augmented2_df], ignore_index=True)
augmented_df_full

,Filename,Label,Augmented_Image
0,Image_10.png,AstonMartin,"(((tf.Tensor(0.16604683, shape=(), dtype=float..."
1,Image_10.png,AstonMartin,"(((tf.Tensor(0.16604683, shape=(), dtype=float..."
2,Image_10.png,AstonMartin,"(((tf.Tensor(0.16604683, shape=(), dtype=float..."
3,Image_10.png,AstonMartin,"(((tf.Tensor(0.16604683, shape=(), dtype=float..."
4,Image_10.png,AstonMartin,"(((tf.Tensor(0.16604683, shape=(), dtype=float..."
...,...,...,...
2655,Porsche.png,Porsche,"(((tf.Tensor(1.0, shape=(), dtype=float32), tf..."
2656,Porsche.png,Porsche,"(((tf.Tensor(1.0, shape=(), dtype=float32), tf..."
2657,Porsche.png,Porsche,"(((tf.Tensor(1.0, shape=(), dtype=float32), tf..."
2658,Porsche.png,Porsche,"(((tf.Tensor(1.0, shape=(), dtype=float32), tf..."


In [19]:
label_encoder = LabelEncoder()
augmented_df_full['Label_coded'] = label_encoder.fit_transform(augmented_df_full['Label'])
augmented_df_full

,Filename,Label,Augmented_Image,Label_coded
0,Image_10.png,AstonMartin,"(((tf.Tensor(0.16604683, shape=(), dtype=float...",0
1,Image_10.png,AstonMartin,"(((tf.Tensor(0.16604683, shape=(), dtype=float...",0
2,Image_10.png,AstonMartin,"(((tf.Tensor(0.16604683, shape=(), dtype=float...",0
3,Image_10.png,AstonMartin,"(((tf.Tensor(0.16604683, shape=(), dtype=float...",0
4,Image_10.png,AstonMartin,"(((tf.Tensor(0.16604683, shape=(), dtype=float...",0
...,...,...,...,...
2655,Porsche.png,Porsche,"(((tf.Tensor(1.0, shape=(), dtype=float32), tf...",13
2656,Porsche.png,Porsche,"(((tf.Tensor(1.0, shape=(), dtype=float32), tf...",13
2657,Porsche.png,Porsche,"(((tf.Tensor(1.0, shape=(), dtype=float32), tf...",13
2658,Porsche.png,Porsche,"(((tf.Tensor(1.0, shape=(), dtype=float32), tf...",13


In [20]:
augmented_df_full['Label_coded'].unique()

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13])

In [21]:
#to check if number matches
augmented_df_full['Label'].unique()

array(['AstonMartin', 'Audi', 'Bentley', 'Bugatti', 'Ferrari',
       'FiskerAutomotive', 'Ford', 'Koenigsegg', 'Lamborghini',
       'LotusCars', 'Maserati', 'Mercedes-AMG', 'Pagani', 'Porsche'],
      dtype=object)

## Prepare for Modelling

In [22]:
#split it into features and labels
images = augmented_df_full.drop(['Filename','Label_coded','Label'], axis=1)
labels = augmented_df_full[['Label_coded']]

In [23]:
X_train ,X_test, y_train, y_test=train_test_split(images, labels, test_size=0.2, random_state=42)

In [ ]:
# Convert PIL images/tensors in DataFrame to NumPy array
X_train = np.array([np.array(img) for img in X_train.iloc[:, 0]])
X_test = np.array([np.array(img) for img in X_test.iloc[:, 0]])

# Reshape the NumPy array to the desired shape
X_train = X_train.reshape(len(X_train), 150, 150, 3)
X_test = X_test.reshape(len(X_test), 150, 150, 3)

y_train=y_train.values
y_test=y_test.values
y_train=y_train.reshape(len(y_train),1)
y_test=y_test.reshape(len(y_test),1)

X_train=X_train/255.0
X_test=X_test/255.0

In [ ]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
#CNN Model
cnn_model_1 = Sequential()

# Convolutional layers with Batch Normalization
cnn_model_1.add(Conv2D(kernel_size=(3, 3), filters=64, input_shape=(150, 150, 3), activation='relu'))
cnn_model_1.add(BatchNormalization())
cnn_model_1.add(MaxPooling2D(pool_size=(2, 2)))

cnn_model_1.add(Conv2D(kernel_size=(3, 3), filters=128, activation='relu'))
cnn_model_1.add(BatchNormalization())
cnn_model_1.add(MaxPooling2D(pool_size=(2, 2)))

cnn_model_1.add(Conv2D(kernel_size=(3, 3), filters=256, activation='relu'))
cnn_model_1.add(BatchNormalization())
cnn_model_1.add(MaxPooling2D(pool_size=(2, 2)))

# Dropout layer
cnn_model_1.add(Dropout(0.5))

# Global average pooling layer
cnn_model_1.add(GlobalAveragePooling2D())

# Output layer
cnn_model_1.add(Dense(14, activation='softmax'))

cnn_model_1.summary()

In [ ]:
#Compile the model 
cnn_model_1.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

#Train the model XX epochs (obviously we need more than 2, maybe 30?)
H = cnn_model_1.fit(X_train, y_train, epochs=2, validation_data=(X_test, y_test))

In [ ]:
print("Evaluate on test data")
results = cnn_model_1.evaluate(X_test, y_test)
print("test loss, test acc:", results)

In [ ]:
plt.plot(H.history['accuracy'], label='accuracy')
plt.plot(H.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1])
plt.legend(loc='lower right')

test_loss, test_acc = cnn_model_1.evaluate(X_test,  y_test, verbose=2)
print(test_acc)

In [ ]:
training_accuracy = H.history['loss']
validation_accuracy = H.history['val_loss']
plt.plot(training_accuracy, 'r', label = 'training loss')
plt.plot(validation_accuracy, 'b', label = 'validation loss')
plt.title('training and test loss')
plt.xlabel('epochs')
plt.ylabel('loss')
plt.legend()
plt.show()

In [ ]:
#predict on test set
predictions = cnn_model_1.predict(X_test)

#create target names
targetnames=augmented_df['Label'].unique()

# convert probability predictions to table using class names for column names
prediction_df = pd.DataFrame(predictions, columns=targetnames)

# inspect 
print(prediction_df.head())

# convert predictions to class labels
predicted_labels = np.argmax(predictions, axis=1)
print(predicted_labels)



In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns

# create a confusion matrix
conf_matrix = confusion_matrix(y_test, predicted_labels)

# Convert to a pandas dataframe
confusion_df = pd.DataFrame(conf_matrix, index=targetnames, columns=targetnames)

# Set the names of the x and y axis, this helps with the readability of the heatmap.
confusion_df.index.name = 'True Label'
confusion_df.columns.name = 'Predicted Label'

sns.heatmap(confusion_df, annot=True, fmt='3g')

## SVM

In [26]:
#split it into features and labels for SVM
images = augmented_df_full.drop(['Filename','Label_coded','Label'], axis=1)
labels = augmented_df_full.Label_coded

In [28]:
X_train ,X_test, y_train, y_test=train_test_split(images, labels, test_size=0.2, random_state=42)

In [29]:
# perform PCA on X to reduce amount of features
# reduce to dimensionality so that 80% of variance is kept
from sklearn.decomposition import PCA

# Convert PIL images/tensors in DataFrame to NumPy array
X_train = np.array([np.array(img) for img in X_train.iloc[:, 0]])
X_test = np.array([np.array(img) for img in X_test.iloc[:, 0]])

# reshape features accordingly to SVM architecture NOTE: train test split again!!
X_train = X_train.reshape((X_train.shape[0], -1))
X_test = X_test.reshape((X_test.shape[0], -1))

pca = PCA(n_components=0.8, random_state=42)
X_pca_train = pca.fit_transform(X_train)

# check how many principal components are needed
pca.n_components_

# apply the trained PCA model on the test set as well
X_pca_test = pca.transform(X_test)

# check that PCA worked
print(X_pca_train.shape)
print(X_pca_test.shape)
print(y_train.shape)
print(y_test.shape)

(2128, 6)
(532, 6)
(2128,)
(532,)


In [30]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

def print_score(clf, X_train, y_train, X_test, y_test, train=True):
    if train:
        pred = clf.predict(X_train)
        clf_report = pd.DataFrame(classification_report(y_train, pred, output_dict=True))
        print("Train Result:\n================================================")
        print(f"Accuracy Score: {accuracy_score(y_train, pred) * 100:.2f}%")
        print("_______________________________________________")
        print(f"CLASSIFICATION REPORT:\n{clf_report}")
        print("_______________________________________________")
        print(f"Confusion Matrix: \n {confusion_matrix(y_train, pred)}\n")
        
    elif train==False:
        pred = clf.predict(X_test)
        clf_report = pd.DataFrame(classification_report(y_test, pred, output_dict=True))
        print("Test Result:\n================================================")        
        print(f"Accuracy Score: {accuracy_score(y_test, pred) * 100:.2f}%")
        print("_______________________________________________")
        print(f"CLASSIFICATION REPORT:\n{clf_report}")
        print("_______________________________________________")
        print(f"Confusion Matrix: \n {confusion_matrix(y_test, pred)}\n")

In [ ]:
from sklearn.svm import SVC

# The hyperparameter coef0 controls how much the model is influenced by high degree ploynomials 
model = SVC(kernel='poly', degree=2, gamma='auto', coef0=1, C=5)
model.fit(X_pca_train, y_train)

In [ ]:
print_score(model, X_pca_train, y_train, X_pca_test, y_test, train=True)
print_score(model, X_pca_train, y_train, X_pca_test, y_test, train=False)

In [ ]:
##or is this better???

In [ ]:
param_grid={'C':[0.1,1,10,100], 
            'gamma':[0.0001,0.001,0.1,1], 
            'kernel':['rbf','poly']} 
  
#SVC internally uses the one-vs-one strategy by default
svc=svm.SVC(probability=True) 
  
#creating a model
model=GridSearchCV(svc,param_grid)
model.fit(X_pca_train,y_train)

In [ ]:
#best parameters obtained from GridSearchCV
model.best_params_